https://www.animeboston.com/schedule/index/2025

# Notes

- Careful for time slots, verify everything lines up (triple check) don't want a repeat of what happened last time Johnny boy
- Also, are there missing events? Verify that all events are accounted for...
- - I am noticing some dropoff of events... Check rows and make sure we are all accounted for. For example original tables

In [166]:
import pandas as pd
import ast  # convert string list

### Notes this is for refreshing and debugging


In [162]:
# Run when refreshing data!!!!!!!
%run WebScraping.py # Refreshes all the original tables

In [184]:
# Run when refreshing data!!!!!!!
# %run WebScraping.py # Refreshes all the original tables

## Last updated (Tables 0, 1, 2)
# Friday, May 23rd, 2025
#   Last Updated: Wed, 05/14/2025, 2:28 pm
# Saturday, May 24th, 2025
#   Last Updated: Thu, 05/15/2025, 6:00 pm
# Sunday, May 25th, 2025
#   Last Updated: Tue, 05/13/2025, 7:45 pm



# #Run when needing to overwrite any new events or whatnot
# categories_JRW = pd.read_csv("Event_Categories_JRW.csv").drop(columns={'Color','Data Value'})
# categories_JRW['Category'] = categories_JRW['Category'].str.strip() # clean merge

# for ii in range(0, 3):
#     df_final = pd.read_csv(f"AnimeBoston_day{ii}_schedule.csv", converters={'TimeSlot': ast.literal_eval})
#     df_final['Category'] = df_final['Category'].str.strip() # clean merge
#     df_merge = pd.merge(df_final, categories_JRW, on='Category', how='left')
#     df_merge.to_csv(f"AnimeBoston_day{ii}_schedule_JRW.csv", index=False, encoding='utf-8-sig')

# dfs = []
# for ii in range(3):# loop over days
#     # read each file
#     df = pd.read_csv(f'AnimeBoston_day{ii}_schedule_JRW.csv')
#     dfs.append(df)

# # combine all days
# df_all = pd.concat(dfs, ignore_index=True)

# # filter NaN Utility
# df_nan_utility = df_all[df_all['Utility_general_JRW'].isna()]
# df_nan_utility

# Optimization Plan
## Preprocessing Algorithm

1. **Filter by global utility**  
   Remove any event $e$ with $U_e < U_{\text{threshold}}$ .

2. **Filter early low-utility**  
   For each event $e$, if $\min T_e < T_{\text{wake}}$ and $U_e < U_{\text{min}}$, remove $e$ .

3. **Build filtered set**  
   Let  
   $$
   E = \{\,e : e\ \text{passes both filters}\,\}.
   $$

4. **Group assignment (if applicable)**  
   Partition $E$ into groups $G$, where each group $g \in G$ is a set $E_g \subseteq E$ of duplicate/rerun events. Each event belongs to exactly one group.

---

## Sets

- $T$ = all 15-minute time slots  
- $E$ = all events after preprocessing  
- $G$ = set of event groups (reruns)  
- $E_g \subseteq E$ = events in group $g$  
- $\mathcal{S}$ = set of unique overlap groups (events sharing the same time slot set)

---

## Parameters

- $U_e \in \mathbb{R}_{\ge 0}$: utility of event $e$  
- $U_{\text{min}}$: minimum utility for early events  
- $U_{\text{threshold}}$: general utility threshold  
- $T_{\text{wake}}$: earliest allowed time for low-utility events

---

## Decision Variables

- $x_e \in \{0, 1\}$ — 1 if event $e$ is selected, 0 otherwise

- Could potentially be real-valued if in totally unimodular format (assignment problem)

---

## Objective

Maximize total utility:
$$
\max \sum_{e \in E} U_e \cdot x_e
$$

---

## Constraints

**(1) No Overlap (Condensed):**  
No two overlapping events can be selected. For each unique overlap group $S \in \mathcal{S}$:
$$
\sum_{e \in S} x_e \le 1 \quad \forall S \in \mathcal{S}
$$

Where each $S$ corresponds to:
$$
S = \{\, e \in E : t \in T_e \,\} \quad \text{for some } t \in T
$$
with duplicates removed across time slots.

<!-- **(2) One per Group (may not apply):**  
At most one event can be selected per group $g \in G$:
$$
\sum_{e \in E_g} x_e \le 1 \quad \forall g \in G
$$ -->

---
## Future improvements
- it should be sequential (or even total, as in it considers each day all in one), and seek not to repeat events over the course of different days, so preprocessing out? or setting a constraint that prior day events are equal to 0.

In [257]:
# params
U_threshold = 5 # base level of Utility
U_min = 7 # sacrificing early sleep of 7 utility
desired_time = pd.to_datetime('09:45', format='%H:%M') # desired arrival time

In [258]:
df_raw = pd.read_csv(f"AnimeBoston_day{0}_schedule.csv", 
                 encoding='utf-8', 
                 converters={'TimeSlot': ast.literal_eval})
df = df_raw[(df_raw['Utility_JW']>U_threshold)]
df

,Event,Room,Utility_JW,Category,Color,TimeSlot
1,25 Great Anime Movies You've Never Heard of,"('Hynes', 'Panel 208')",6.00,Panel,#72B7ED,"[16:00, 16:15, 16:30, 16:45]"
3,30 Years Ago: Anime in 1995,"('Hynes', 'Panel 207')",6.00,Guest Content,#00FF00,"[12:30, 12:45, 13:00, 13:15]"
7,A Cozy Conversation with Jill Harris & Kana Ic...,"('Sheraton', 'Grand Ballroom')",9.00,Guest Content,#00FF00,"[14:00, 14:15, 14:30, 14:45]"
9,A Look at Anime in the 90s,"('Hynes', 'Panel 310')",6.00,Anime,#FFFF00,"[12:00, 12:15, 12:30, 12:45]"
10,A Sophisticatedly Unsophisticated Look At Fan ...,"('Hynes', 'Panel 207')",7.00,Panel,#72B7ED,"[23:00, 23:15, 23:30, 23:45]"
11,AMV Contest,"('Hynes', 'Ballroom B')",10.00,Fan Creations|Video,#66A3D2|#CCFF66,"[21:00, 21:15, 21:30, 21:45, 22:00, 22:15, 22:..."
12,AMV Genkis,"('Hynes', 'Fan Creations 312')",5.50,Fan Creations,#CCFF66,"[09:00, 09:15, 09:30, 09:45]"
13,AMV Sing-A-Long,"('Hynes', 'Fan Creations 312')",6.00,Video,#66A3D2,"[13:00, 13:15, 13:30, 13:45]"
15,All the Feels,"('Hynes', 'Fan Creations 312')",10.00,Fan Creations,#CCFF66,"[17:30, 17:45, 18:00, 18:15]"
17,Anime Jeopardy!,"('Hynes', 'Panel 310')",7.00,Gameshow,#72B7ED,"[19:30, 19:45, 20:00, 20:15]"


In [259]:
# Step 1: Copy and extract start/end
df = df.copy()
df['start_time'] = df['TimeSlot'].str[0]
df['end_time'] = df['TimeSlot'].str[-1]

# Step 2: Convert start_time to datetime, assume dummy date
df['start_dt'] = pd.to_datetime(df['start_time'], format='%H:%M')

# Step 3: Adjust early-morning times (e.g., < 06:00) to next day
df['start_dt'] = df['start_dt'].apply(lambda t: t + pd.Timedelta(days=1) if t.hour < 6 else t)

# Step 4: Sort and clean up
df = df.sort_values('start_dt').reset_index(drop=True)
df = df.drop(columns=['start_dt'])

# convert to datetime
df['start_dt'] = pd.to_datetime(df['start_time'], format='%H:%M')

# apply conditional filter
df_final = df[(df['start_dt'] > desired_time) | ((df['start_dt'] <= desired_time) & (df['Utility_JW'] >= U_min))]

# optional: drop helper column
df_final = df_final.drop(columns=['start_dt'])
df_final

,Event,Room,Utility_JW,Category,Color,TimeSlot,start_time,end_time
1,Original Fan Animation,"('Hynes', 'Fan Creations 312')",10.00,Fan Creations,#CCFF66,"[10:00, 10:15, 10:30, 10:45, 11:00, 11:15]",10:00,11:15
2,Opening Ceremonies,"('Hynes', 'Memorial Auditorium')",9.50,Stage Show,#AFE1F1,"[10:00, 10:15, 10:30, 10:45, 11:00, 11:15]",10:00,11:15
3,The History of Digimon in America,"('Sheraton', 'Panel Constitution A')",8.00,Fan Content,#FF69B4,"[10:00, 10:15, 10:30, 10:45]",10:00,10:45
4,Cosplay Bingo,"('Hynes', 'Events 302')",6.00,Cosplay,#CC99FF,"[11:00, 11:15, 11:30, 11:45]",11:00,11:45
5,Jam Zone: Shiori Kubrick,"('Sheraton', 'Panel Republic')",6.00,Music,#CCFFFF,"[11:00, 11:15, 11:30, 11:45]",11:00,11:45
6,A Look at Anime in the 90s,"('Hynes', 'Panel 310')",6.00,Anime,#FFFF00,"[12:00, 12:15, 12:30, 12:45]",12:00,12:45
7,K-Pop Dance Workshop,"('Hynes', 'Ballroom A')",6.00,Partner Content,#FF69B4,"[12:00, 12:15, 12:30, 12:45]",12:00,12:45
8,30 Years Ago: Anime in 1995,"('Hynes', 'Panel 207')",6.00,Guest Content,#00FF00,"[12:30, 12:45, 13:00, 13:15]",12:30,13:15
9,Anime On Trial,"('Sheraton', 'Panel Constitution B')",6.50,Stage Show,#AFE1F1,"[12:30, 12:45, 13:00, 13:15]",12:30,13:15
10,Meet Kaiji Tang,"('Sheraton', 'Panel Constitution A')",6.00,Guest Content,#00FF00,"[13:00, 13:15, 13:30, 13:45]",13:00,13:45


### What's next

Ideally would use an index instead of the event names, this way it is easier to join and merge



In [261]:
df_final['EventID'] = df_final.index

# getting constraints sets
df_final_exploded = df_final.explode('TimeSlot').sort_values(by=['Room', 'Event']).reset_index(drop=True)
df_final_exploded

,Event,Room,Utility_JW,Category,Color,TimeSlot,start_time,end_time,EventID
0,K-Pop Dance Workshop,"('Hynes', 'Ballroom A')",6.0,Partner Content,#FF69B4,12:00,12:00,12:45,7
1,K-Pop Dance Workshop,"('Hynes', 'Ballroom A')",6.0,Partner Content,#FF69B4,12:15,12:00,12:45,7
2,K-Pop Dance Workshop,"('Hynes', 'Ballroom A')",6.0,Partner Content,#FF69B4,12:30,12:00,12:45,7
3,K-Pop Dance Workshop,"('Hynes', 'Ballroom A')",6.0,Partner Content,#FF69B4,12:45,12:00,12:45,7
4,AMV Contest,"('Hynes', 'Ballroom B')",10.0,Fan Creations|Video,#66A3D2|#CCFF66,21:00,21:00,23:15,36
...,...,...,...,...,...,...,...,...,...
200,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",6.5,Social Event|Workshop,#F7931E,17:45,17:30,18:45,24
201,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",6.5,Social Event|Workshop,#F7931E,18:00,17:30,18:45,24
202,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",6.5,Social Event|Workshop,#F7931E,18:15,17:30,18:45,24
203,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",6.5,Social Event|Workshop,#F7931E,18:30,17:30,18:45,24


In [263]:
df_final_timeslotgrouped = df_final_exploded.groupby(['TimeSlot'])[['EventID']].agg(set).reset_index()
df_final_timeslotgrouped

,TimeSlot,EventID
0,00:00,{39}
1,00:15,{39}
2,01:00,{40}
3,01:15,{40}
4,01:30,{40}
5,01:45,{40}
6,10:00,"{1, 2, 3}"
7,10:15,"{1, 2, 3}"
8,10:30,"{1, 2, 3}"
9,10:45,"{1, 2, 3}"


In [266]:
# keep only first duplicates
# make Event lists hashable for deduplication
df_final_timeslotgrouped["Event_tuple"] = df_final_timeslotgrouped["EventID"].apply(tuple)

# get length of each tuple
df_final_timeslotgrouped["Event_tuple_length"] = df_final_timeslotgrouped["Event_tuple"].apply(len)

# drop duplicates based on the Event list only, keeping the first time slot
df_dedupped_by_event = df_final_timeslotgrouped.drop_duplicates(subset="Event_tuple", keep="first")

# drop the helper column
df_dedupped_by_event = df_dedupped_by_event.drop(columns="Event_tuple").reset_index().drop(columns=['index'])
df_set_of_constraints_one = df_dedupped_by_event[df_dedupped_by_event['Event_tuple_length']>1].reset_index().drop(columns=['index'])
df_set_of_constraints_one.rename(columns={'EventID':'OverlappingEventIDs'}, inplace=True)
df_set_of_constraints_one

,TimeSlot,OverlappingEventIDs,Event_tuple_length
0,10:00,"{1, 2, 3}",3
1,11:00,"{1, 2, 4, 5}",4
2,11:30,"{4, 5}",2
3,12:00,"{6, 7}",2
4,12:30,"{8, 9, 6, 7}",4
5,13:00,"{8, 9, 10, 11, 12, 13}",6
6,13:30,"{10, 11, 12, 13, 14}",5
7,14:00,"{13, 14, 15, 16, 17}",5
8,14:15,"{13, 14, 15, 16, 17, 18}",6
9,14:30,"{15, 16, 17, 18, 19}",5
